In [2]:
pip install duckdb


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Standard imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # plotting library
import pandas as pd

## Package for sql
import duckdb

## Distribution fitting
# from scipy import stats

In [4]:
#import dataframe

df = pd.read_parquet('/home/joelva/code/pgt-group-3/data/raw/Complete_Set/HECTORdataAggregated-2.parquet')


describe_column(df, 'ToDisp_Drive_Temp_Motor_y')

In [ ]:
# Data cleaning
# Convert 'Bat_MaximumVoltage_x' column to datetime if it's not already in datetime format
df['Bat_MaximumVoltage_x'] = pd.to_datetime(df['Bat_MaximumVoltage_x'])

# Data cleaning
df['Bat_TotalCurrent_y'] = df['Bat_TotalCurrent_y'].replace(-3200.0, 0)
df['Chas_Weight_Axle_1_y'] = df['Chas_Weight_Axle_1_y'].replace(32767.0, 0)
df['Chas_Weight_Axle_2_y'] = df['Chas_Weight_Axle_2_y'].replace(32767.0, 0)
df['Chas_Weight_Axle_3_y'] = df['Chas_Weight_Axle_3_y'].replace(32767.0, 0)

# Remove column
del df['Chas_Weight_Total_y']

# Negative fuels cell values? For now we keep it in the data
# FCell_RX_Stack_Current_y
# FCell_2_RX_Stack_Current_y
# FCell_3_RX_Stack_Current_y

# Negative torque? Engine braking?
# MINVa_RX_VP_Status1_1_ActualTorque_y

# Replace specific values (-40.0 in this case) with NaN
df['ToDisp_Drive_Temp_Motor_y'] = df['ToDisp_Drive_Temp_Motor_y'].replace(-40.0, np.nan)

# Forward fill only for the positions where the value was -40.0
df['ToDisp_Drive_Temp_Motor_y'] = df['ToDisp_Drive_Temp_Motor_y'].fillna(method='ffill')

In [ ]:
# Feature engineering
# Create a column 'BattOutputPower' calculated with V*I
df['BattOutputPower'] = df['Bat_TotalCurrent_y'] * df['Bat_TotalVoltage_y']

# Create a column 'MotorPowerDraw' calculated with V*I
df['MotorPowerDraw'] = df['MINVa_RX_VP_Status2_1_ActualcalcDC_INCurrent_y'] * df['Bat_TotalVoltage_y']

# Create a column 'FCPowerDelivery' calculated with the sum of the V*I from the three Fuel Cells
df['TotalFCPower'] = df['FCell_2_RX_Stack_Current_y'] * df['FCell_2_RX_Stack_Voltage_y'] + df['FCell_3_RX_Stack_Current_y'] * df['FCell_3_RX_Stack_Voltage_y'] + df['FCell_RX_Stack_Current_y'] * df['FCell_RX_Stack_Voltage_y']

# Create a column 'ActualSpeed' calculated MotorRPM * 0.175
df['ActualSpeed'] = df['MINVa_RX_VP_Status1_1_ActualSpeed_y'] * 0.175

# Create a column calculating the total power demand by including the compactor system
# this is calculated by adding the power draw of the compactor system (*10) to the power draw of the motor
df['TotalPowerDraw'] = df['Data_Energy_CurrentEnergyBody_y'] * 10 + df['MotorPowerDraw']



In [ ]:
#Renaming columns with a more friendly
column_rename_mapping = {
    'Bat_MaximumVoltage_x': 'datetime',
    'Bat_MaximumVoltage_y': 'BatMaxV',
    'Bat_MinimumVoltage_y': 'BatMinV',
    'Bat_SOC_y': 'BatSOC',
    'Bat_TotalCurrent_y': 'BatTotalCurrent',
    'Bat_TotalVoltage_y': 'BatTotalVoltage',
    'Body_Active_y': 'CompOnOff',
    'Body_EmergencyStopActivated_y': 'StopOnOff',
    'Chas_AccPedalPos1000_y': 'ThrottlePerc',
    'Chas_BrakePedalPos1000_y': 'BrakePerc',
    'Chas_Signal_AirCon_SW_y': 'Signal',
    'Chas_TotalVehicleDistance_m_y': 'TotalDistance',
    'Chas_Weight_Axle_1_y': 'FrontAxle',
    'Chas_Weight_Axle_2_y': 'RearAxle1',
    'Chas_Weight_Axle_3_y': 'RearAxle2',
    'Chas_Weight_Total_y': 'TotalAxle',
    'Data_Energy_CurrentEnergyBody_y': 'CompPower',
    'FCell_2_RX_air_flow_y': 'FC2AirFlow',
    'FCell_2_RX_cool_Temp_y': 'FC2Cool',
    'FCell_2_RX_Stack_Current_y': 'FC2Current',
    'FCell_2_RX_Stack_Voltage_y': 'FC2Volt',
    'FCell_3_RX_air_flow_y': 'FC3AirFlow',
    'FCell_3_RX_cool_Temp_y': 'FC3Cool',
    'FCell_3_RX_Stack_Current_y': 'FC3Current',
    'FCell_3_RX_Stack_Voltage_y': 'FC3Volt',
    'FCell_RX_air_flow_y': 'FC1AirFlow',
    'FCell_RX_cool_Temp_y': 'FC1Cool',
    'FCell_RX_Stack_Current_y': 'FC1Current',
    'FCell_RX_Stack_Voltage_y': 'FC1Volt',
    'gps_altitude_y': 'GPSAltitude',
    'gps_latitude_y': 'GPSLatitude',
    'gps_longitude_y': 'GPSLongitude',
    'H2_Fill_ALL_y': 'H2Perc',
    'H2_Press_ALL_y': 'H2Press',
    'H2_Temp_ALL_y': 'H2Temp',
    'H2_Weight_ALL_y': 'H2Weight',
    'HVS3_RX_Result_I_Value_y': 'HVACDraw',
    'MINVa_RX_VP_Status1_1_ActualSpeed_y': 'MotorRPM',
    'MINVa_RX_VP_Status1_1_ActualTorque_y': 'MotorTorque',
    'MINVa_RX_VP_Status2_1_ActualcalcDC_INCurrent_y': 'MotorUseCurrent',
    'ToDisp_Drive_Temp_Motor_y': 'MotorTemp',
    'VBB_30_y': '24VBatt'
}
df.rename(columns=column_rename_mapping, inplace=True)

Deze functie, 'aggregate', is ontworpen om een SQL-query samen te stellen voor het aggregeren van gegevens uit een DataFrame met behulp van DuckDB. De functie creëert een query die verschillende aggregatiefuncties toepast, zoals het berekenen van gemiddelden (AVG) en het vinden van maximale waarden (MAX), voor een reeks gespecificeerde kolommen. De gegevens worden gegroepeerd op basis van een afgekorte versie van de 'datetime'-kolom en geordend op basis van deze gegroepeerde tijd. Dit is bijzonder nuttig voor het analyseren van tijdsgebonden data, waarbij men inzicht wil krijgen in trends over verschillende tijdsintervallen. De output van de functie is een nieuw DataFrame dat de geaggregeerde resultaten bevat, wat handig is voor verdere analyse of visualisatie.

In [1]:
#QUERY
def aggregate():
    # Basis SELECT statement voor de SQL-query
    v_select = "SELECT " 

    # Definieer de kolommen die moeten worden opgenomen in de query.
    # Dit omvat berekeningen zoals AVG (gemiddelde) en MAX (maximale waarde) van de gespecificeerde kolommen.
    v_col0 = "LEFT(Cast(t.datetime AS VARCHAR), 18) AS Vdatetime,"
    v_col1 = "MAX(t.datetime) AS datetime,"
    v_col2 = "AVG(t.BatMaxV) AS BatMaxV,"
    v_col3 = "AVG(t.BatMinV) AS BatMinV,"
    v_col4 = "AVG(t.BatSOC) AS BatSOC,"
    v_col5 = "AVG(t.BatTotalCurrent) AS BatTotalCurrent,"
    v_col6 = "AVG(t.BatTotalVoltage) AS BatTotalVoltage,"
    v_col7 = "MAX(t.CompOnOff) AS CompOnOff,"
    v_col8 = "MAX(t.StopOnOff) AS StopOnOff,"
    v_col9 = "AVG(t.ThrottlePerc) AS ThrottlePerc,"
    v_col10 = "AVG(t.BrakePerc) AS BrakePerc,"
    v_col11 = "MAX(t.Signal) AS Signal,"
    v_col12 = "AVG(t.TotalDistance) AS TotalDistance,"
    v_col13 = "AVG(t.FrontAxle) AS FrontAxle,"
    v_col14 = "AVG(t.RearAxle1) AS RearAxle1,"
    v_col15 = "AVG(t.RearAxle2) AS RearAxle2,"
    v_col16 = "AVG(t.CompPower) AS CompPower,"
    v_col17 = "AVG(t.FC2AirFlow) AS FC2AirFlow,"
    v_col18 = "AVG(t.FC2Cool) AS FC2Cool,"
    v_col19 = "AVG(t.FC2Current) AS FC2Current,"
    v_col20 = "AVG(t.FC2Volt) AS FC2Volt,"
    v_col21 = "AVG(t.FC3AirFlow) AS FC3AirFlow,"
    v_col22 = "AVG(t.FC3Cool) AS FC3Cool,"
    v_col23 = "AVG(t.FC3Current) AS FC3Current,"
    v_col24 = "AVG(t.FC3Volt) AS FC3Volt,"
    v_col25 = "AVG(t.FC1AirFlow) AS FC1AirFlow,"
    v_col26 = "AVG(t.FC1Cool) AS FC1Cool,"
    v_col27 = "AVG(t.FC1Current) AS FC1Current,"
    v_col28 = "AVG(t.FC1Volt) AS FC1Volt,"
    v_col29 = "MAX(t.GPSAltitude) AS GPSAltitude,"
    v_col30 = "MAX(t.GPSLatitude) AS GPSLatitude,"
    v_col31 = "MAX(t.GPSLongitude) AS GPSLongitude,"
    v_col32 = "AVG(t.H2Perc) AS H2Perc,"
    v_col33 = "AVG(t.H2Press) AS H2Press,"
    v_col34 = "AVG(t.H2Temp) AS H2Temp,"
    v_col35 = "AVG(t.H2Weight) AS H2Weight,"
    v_col36 = "AVG(t.HVACDraw) AS HVACDraw,"
    v_col37 = "AVG(t.MotorRPM) AS MotorRPM,"
    v_col38 = "AVG(t.MotorTorque) AS MotorTorque,"
    v_col39 = "AVG(t.MotorUseCurrent) AS MotorUseCurrent,"
    v_col40 = "AVG(t.MotorTemp) AS MotorTemp,"
    #v_col41 = "AVG(t.24VBatt) AS 24VBatt,"
    v_col42 = "AVG(t.BattOutputPower) AS BattOutputPower,"
    v_col43 = "AVG(t.MotorPowerDraw) AS MotorPowerDraw,"
    v_col44 = "AVG(t.TotalFCPower) AS TotalFCPower,"
    v_col45 = "AVG(t.ActualSpeed) AS ActualSpeed,"
    v_col46 = "AVG(t.TotalPowerDraw) AS TotalPowerDraw"

    # FROM clausule specificeert de data source (in dit geval een DataFrame)
    v_from = " FROM df t "
    
    # GROUP BY clausule om de data te groeperen op een afgekorte versie van de datetime kolom
    v_groupby = "GROUP  BY LEFT(Cast(datetime AS VARCHAR), 18) "

    # ORDER BY clausule om de resultaten te ordenen
    v_orderby = "ORDER  BY 1"

    # Combineert alle delen van de SQL-query in één string
    v_query = v_select + v_col0 + v_col1 + v_col2 + v_col3 + v_col4 + v_col5 + v_col6 + v_col7 + v_col8 + v_col9 + v_col10 + v_col11 + v_col12 + v_col13 + v_col14 + v_col15 + v_col16 + v_col17 + v_col18 + v_col19 + v_col20 + v_col21 + v_col22 + v_col23 + v_col24 + v_col25 + v_col26 + v_col27 + v_col28 + v_col29 + v_col30 + v_col31 + v_col32 + v_col33 + v_col34 + v_col35 + v_col36 + v_col37 + v_col38 + v_col39 + v_col40 + v_col42 + v_col43 + v_col44 + v_col45 + v_col46 + v_from + v_groupby + v_orderby
    
    # Voert de query uit met DuckDB en retourneert het resultaat als een DataFrame
    return duckdb.query(v_query).df()

In [9]:
df = aggregate()

: 

In [ ]:
df.to_parquet('/dbfs/mnt/iqbsdata/HAN/Month2/HECTORdataAggregated.parquet')